### Few-shot learning testing

Here we explore classifiers trained to decode the object shape with varying amounts of training data, from a range of layers within the model.

Tested with tensorflow 2.11.0 and Python 3.10.9.

In [ ]:
import json
import zipfile
import os
import psutil
import numpy as np
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from PIL import Image, ImageOps
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from tensorflow.keras import Model, Sequential, metrics, optimizers, layers
from tensorflow.python.framework.ops import disable_eager_execution
from utils import load_tfds_dataset
from generative_model import encoder_network_large, decoder_network_large, VAE, build_encoder_decoder_large
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

tf.keras.utils.set_random_seed(123)

#### Load model

In [ ]:
K.set_image_data_format('channels_last')

latent_dim = 20
input_shape = (64, 64, 3)

encoder, decoder = build_encoder_decoder_large(latent_dim=latent_dim)
vae = VAE(encoder, decoder, 1)

vae.encoder.load_weights("model_weights/shapes3d_encoder.h5")
vae.decoder.load_weights("model_weights/shapes3d_decoder.h5")

#### Train classifier

(Identical method to classifier used to measure decoding accuracy over time.)

In [ ]:
train_ds, test_ds, train_labels, test_labels = load_tfds_dataset('shapes3d', labels=True, 
                                                                 key_dict= {'shapes3d': 'label_shape'})
train_ds = train_ds / 255
test_ds = test_ds / 255

Get names of intermediate layers:

In [ ]:
for layer in vae.encoder.layers:
    print(layer.name)

In [ ]:
# define number of examples to use
num_examples = list([10, 20, 50, 100, 200, 300, 400, 500, 1000])

# place to store the scores
scores = []

# specify the layers to consider
layers_to_consider = ['input_1', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'mean']  # fill this list with the names of the layers you want to consider

# for each layer in the VAE encoder
for layer in layers_to_consider:

    # define a new model that outputs the intermediate representations
    intermediate_layer_model = Model(inputs=vae.encoder.input, 
                                     outputs=vae.encoder.get_layer(layer).output)

    # create a list to store the scores for this layer
    layer_scores = []

    # for each number of examples
    for num in num_examples:
        # get a subset of the training data
        train_subset = train_ds[:num]
        train_labels_subset = train_labels[:num]
    
        # skip if less than two classes are present in the subset
        if len(np.unique(train_labels_subset)) < 2:
            layer_scores.append(np.nan)
            continue

        # get the intermediate representations
        intermediate_train = intermediate_layer_model.predict(train_subset)
    
        # flatten the intermediate representations
        intermediate_train = np.reshape(intermediate_train, (intermediate_train.shape[0], -1))
    
        # train the classifier
        clf = make_pipeline(StandardScaler(), SVC(probability=True))
        clf.fit(intermediate_train, train_labels_subset)
    
        # get the intermediate representations for the test data
        intermediate_test = intermediate_layer_model.predict(test_ds)
    
        # flatten the intermediate representations
        intermediate_test = np.reshape(intermediate_test, (intermediate_test.shape[0], -1))
    
        # score the classifier
        score = clf.score(intermediate_test, test_labels)
    
        # store the score
        layer_scores.append(score)
    
    # store the scores for this layer
    scores.append(layer_scores)

In [ ]:
layer_name_dict = {'input_1': 'Input',
                   'conv2d': '1st conv. layer',
                   'conv2d_1': '2nd conv. layer',
                   'conv2d_2': '3rd conv. layer',
                   'conv2d_3': '4th conv. layer',
                   'mean': 'Latent code',}

In [ ]:
# plot the scores
plt.figure(figsize=(10, 6))
for i, layer_scores in enumerate(scores):
    plt.plot(num_examples, layer_scores, label=layer_name_dict[layers_to_consider[i]])
plt.xlabel('Number of training examples', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Accuracy', fontsize=16)
plt.title('Classifier performance based on the number of training examples', fontsize=16)
plt.legend(fontsize=14)
plt.savefig('few_shot_testing.png', bbox_inches='tight', dpi=300)
plt.show()